# PromptLayer

>[PromptLayer](https://docs.promptlayer.com/introduction) 是一个提示工程平台。它还有助于LLM可观测性，可视化请求，版本化提示，并追踪使用情况。
>
>虽然`PromptLayer`确实有直接与LangChain集成的LLM（例如[`PromptLayerOpenAI`](/docs/integrations/llms/promptlayer_openai)），但使用回调是将`PromptLayer`与LangChain集成的推荐方式。

在本指南中，我们将介绍如何设置`PromptLayerCallbackHandler`。

有关更多信息，请参阅[PromptLayer文档](https://docs.promptlayer.com/languages/langchain)。

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  langchain-community promptlayer --upgrade

### 获取API凭证

如果您没有PromptLayer账户，请在[promptlayer.com](https://www.promptlayer.com)创建一个。然后通过点击导航栏中的设置图标获取API密钥，
并将其设置为名为`PROMPTLAYER_API_KEY`的环境变量


## 使用方法

开始使用`PromptLayerCallbackHandler`相当简单，它接受两个可选参数：
1. `pl_tags` - 可选的字符串列表，将在PromptLayer上作为标签追踪。
2. `pl_id_callback` - 可选函数，将`promptlayer_request_id`作为参数。此ID可与PromptLayer的所有跟踪功能一起使用，以跟踪元数据、分数和提示使用情况。

## 简单OpenAI示例

在这个简单示例中，我们将`PromptLayerCallbackHandler`与`ChatOpenAI`一起使用。我们添加了一个名为`chatopenai`的PromptLayer标签

In [ ]:
import promptlayer  # 别忘了这个 🍰
from langchain_community.callbacks.promptlayer_callback import (
    PromptLayerCallbackHandler,
)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    temperature=0,
    callbacks=[PromptLayerCallbackHandler(pl_tags=["chatopenai"])],
)
llm_results = chat_llm.invoke(
    [
        HumanMessage(content="1,2,3之后是什么？"),
        HumanMessage(content="再讲个笑话？"),
    ]
)
print(llm_results)

## GPT4All示例

In [ ]:
from langchain_community.llms import GPT4All

model = GPT4All(model="./models/gpt4all-model.bin", n_ctx=512, n_threads=8)
callbacks = [PromptLayerCallbackHandler(pl_tags=["langchain", "gpt4all"])]

response = model.invoke(
    "从前有一次，",
    config={"callbacks": callbacks},
)

## 完整功能示例

在这个示例中，我们解锁了`PromptLayer`的更多功能。

PromptLayer允许您以可视化方式创建、版本化和跟踪提示模板。使用[提示注册表](https://docs.promptlayer.com/features/prompt-registry)，我们可以以编程方式获取名为`example`的提示模板。

我们还定义了一个`pl_id_callback`函数，该函数接收`promptlayer_request_id`并记录分数、元数据，并链接使用的提示模板。在[我们的文档](https://docs.promptlayer.com/features/prompt-history/request-id)中了解有关跟踪的更多信息。

In [ ]:
from langchain_openai import OpenAI


def pl_id_callback(promptlayer_request_id):
    print("prompt layer id ", promptlayer_request_id)
    promptlayer.track.score(
        request_id=promptlayer_request_id, score=100
    )  # 分数是0-100之间的整数
    promptlayer.track.metadata(
        request_id=promptlayer_request_id, metadata={"foo": "bar"}
    )  # 元数据是在PromptLayer上跟踪的键值对字典
    promptlayer.track.prompt(
        request_id=promptlayer_request_id,
        prompt_name="example",
        prompt_input_variables={"product": "烤面包机"},
        version=1,
    )  # 将请求链接到提示模板


openai_llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    callbacks=[PromptLayerCallbackHandler(pl_id_callback=pl_id_callback)],
)

example_prompt = promptlayer.prompts.get("example", version=1, langchain=True)
openai_llm.invoke(example_prompt.format(product="烤面包机"))

就是这么简单！设置完成后，所有请求将显示在PromptLayer仪表板上。
此回调也适用于LangChain上实现的任何LLM。